# Advanced Modeling for Titanic Survival Prediction

After establishing a baseline understanding of the Titanic dataset through exploratory data analysis and initial predictive modeling, the next step involves applying advanced machine learning algorithms to enhance predictive performance and uncover deeper patterns in the data. Survival on the Titanic was influenced by a variety of interacting factors—such as passenger class, age, gender, and family connections—many of which are non-linear and not easily captured by simplistic models.

In this advanced modeling phase, we implement and compare the effectiveness of several powerful classification algorithms:

* **Logistic Regression**
* **Random Forest Classifier**
* **XGBoost Classifier**
* **Support Vector Machine (SVM)**

Each of these models brings unique strengths to the table. By comparing these models across evaluation metrics such as accuracy, precision, recall, and ROC-AUC, we aim to identify the most suitable approach for accurately predicting passenger survival. The outcome of this analysis will not only refine the predictive model but also provide insights into the critical factors that contributed to survival, shedding light on one of the most historically significant maritime disasters through the lens of modern data science.

# Import Libraries

In [1]:
# data structure and visualization libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# machine learning libraries
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# for saving and loading models
import pickle
import joblib
import datetime

# Load and Preapre Data for Modeling

## Load the dataset

In [2]:
#  read cleaned Titanic dataset
df = pd.read_csv("data/titanic_clean.csv")

In [3]:
df.head()

,survived,pclass,fare,alone,sex_male,who_man,who_woman,embark_town_Queenstown,embark_town_Southampton
0,0,3,7.2500,False,True,True,False,False,True
1,1,1,71.2833,False,False,False,True,False,False
2,1,3,7.9250,True,False,False,True,False,True
3,1,1,53.1000,False,False,False,True,False,True
4,0,3,8.0500,True,True,True,False,False,True


## Split the dataset into input features (x) and target variable (y)

In [5]:
# inpurt features
x = df.drop(columns=['survived'], axis=1)

# target variable
y = df['survived']

In [6]:
# check features and target variable
print("Features:\n", x.head())
print()
print("\nTarget Variable:\n", y.head())

Features:
    pclass     fare  alone  sex_male  who_man  who_woman  \
0       3   7.2500  False      True     True      False   
1       1  71.2833  False     False    False       True   
2       3   7.9250   True     False    False       True   
3       1  53.1000  False     False    False       True   
4       3   8.0500   True      True     True      False   

   embark_town_Queenstown  embark_town_Southampton  
0                   False                     True  
1                   False                    False  
2                   False                     True  
3                   False                     True  
4                   False                     True  


Target Variable:
 0    0
1    1
2    1
3    1
4    0
Name: survived, dtype: int64


## Split the dataset into training and testing sets

In [7]:
# split data into training and testing sets (70% train, 30% test)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=42)